<a href="https://colab.research.google.com/github/vpariza/open-hummingbird-eval/blob/main/examples/hbird_eval_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Example on how to evaluate a vision encoder with the Hummingbird or Dense NN Retrieval Evaluation

<a href="https://githubtocolab.com/vpariza/open-hummingbird-eval/blob/main/examples/hbird_eval_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### 1. Install required Libraries

In [ ]:
!pip install torchmetrics
!pip install scann
# Restart your session after installing the packages

### 2. Clone our repository

In [ ]:
!git clone https://github.com/vpariza/open-hummingbird-eval.git

In [ ]:
# Move to the repository folder
%cd open-hummingbird-eval

### 3. Download your preferred dataset

In [ ]:
# Used to delete previously downloaded and used
# !rm -rf ./VOCSegmentation
# !rm voc_data.zip

#### Option 1: Tiny Pascal VOC

In [ ]:
# Download the tiny Pascal VOC - https://1drv.ms/u/s!AnBBK4_o1T9MdIuhcH4gbjsTdTY?e=spmlzg # tiny
!wget -O voc_data.zip "https://cpf5rw.am.files.1drv.com/y4mxW5pUDPP2WOVWXZYdhd5PK82qXcqQURFvVTXGSfEk8igihjx8oyA_iSmeuMnwLSyNwN601Jq9Ec9PyN3olpCCCoNtrgYg7DkvrXUXyI-mymjQiSR0kgRZDvRRUw4SD9i4QFTa-q5W_A7WahXU5v1XroWUU8bKDgUbY0xbBuX67xZr4HGDQiT5b0cr4iPQbt6NJlKkAyWdXUkIL37xpL3JQ"

#### Option 2: Mini Pascal VOC

In [ ]:
# Download the mini Pascal VOC - https://1drv.ms/u/s!AnBBK4_o1T9MdS8wbopnWowJcpM?e=VHhsFB # mini
!wget -O voc_data.zip "https://cphe0g.am.files.1drv.com/y4mUV45Lrel1tdHEu_hHhs_MlkJRu7UMJs_cfkPYHyg1tIYVfUwTh9kfATfcHuhrsv8rgXQ36QJ6JRR484GCSQ3n47nAngLCc_-cXJK-QH0Lu8oVJvzsioRdHIug3CdgLWNeLt3blNMs9U3HB6WR0R9GmiGJG-G0MfDg4DoQYA-dXu2LYUGLrBME_4REFDemURh0Gptvys0UGWKN5cPi43Zsw"

#### Option 3: Pascal VOC

In [ ]:
# Download the full Pascal VOC - https://1drv.ms/u/s!AnBBK4_o1T9MbXrxhV7BpGdS8tk?e=P7G6F0 # full
!wget -O voc_data.zip "https://c2ymfq.am.files.1drv.com/y4mZqATCHOv_Z88obTJ_ZatMGDEx6ts5TzOyJnVLKqmkXZwdL_PKIMFLNmZR9FLFJ1CHMC6h_7bJjxlcUM8yXjG92Ms1X-95x6Dh90QgawSpYDPoE1gmLx3VwnW2amZZEog-omFd87fTKZqn3lpP_mtisDQsfDzruBgz_JHcSWsZd2jrsN2qoV3cJ5HammjY_im0ftfjwNOup1EuiWcQ9KT6A"

### 4. Unzip Contents of zip Dataset

In [ ]:
!unzip voc_data.zip

### 5. Evaluate a preferred model on the downloaded dataset

In [ ]:
import torch
from src.hbird_eval import hbird_evaluation

In [ ]:
# Parameters for the model dino
device = 'cuda'
input_size = 224
batch_size = 64
patch_size = 16
embed_dim = 384
model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

In [ ]:
fn = lambda model, imgs: (model.get_intermediate_layers(imgs)[0][:, 1:], None)

In [ ]:
hbird_miou = hbird_evaluation(model.to(device),
        d_model=embed_dim,        # size of the embedding feature vectors of patches
        patch_size=patch_size,
        batch_size = batch_size,
        input_size=224,
        augmentation_epoch=1,     # how many iterations of augmentations to use on top of the training dataset in order to generate the memory
        device=device,
        return_knn_details=False, # whether to return additional NNs details
        n_neighbours=30,         # the number of neighbors to fetch per image patch
        nn_params=None,           # Other parameters to be used for the k-NN operator
        ftr_extr_fn=fn,           # function that extracts features from a vision encoder on images
        dataset_name='voc',       # the name of the dataset to use, currently only Pascal VOC is included.
        data_dir='./VOCSegmentation',    # path to the dataset to use for evaluation
        memory_size=None)
print('Hummingbird Evaluation (mIoU):', hbird_miou)